# Deep Research Agent with OpenHands SDK

This notebook demonstrates a **3-phase deep research workflow**:

| Phase | Agent | Output |
|-------|-------|--------|
| **1. Planning** | GPT-4o creates plan → GPT-5.1 critiques → iterate until approved | `research_plan.md` |
| **2. Research** | GPT-4o executes searches, gathers raw findings | Updated `research_plan.md` |
| **3. Synthesis** | GPT-5.1 synthesizes all findings into comprehensive report | `research_report.md` |

**Features:**
- ✅ Multi-agent collaboration (GPT-4o + GPT-5.1)
- ✅ Iterative plan refinement with scoring
- ✅ **Checkpointing** - Resume from crash/interrupt
- ✅ **Observability** - Laminar tracing (set `LMNR_PROJECT_API_KEY`)
- ✅ Professional report synthesis with citations

## Step 1: Setup (same as notebook 00)

In [1]:
#!pip install openhands-sdk python-dotenv tavily-python -q

In [2]:
import os
import json
import time
from dotenv import load_dotenv
load_dotenv()

from typing import List
from pydantic import Field
from tavily import TavilyClient
from openhands.sdk import LLM, Agent, Conversation, Tool, Action, Observation, ToolDefinition, TextContent
from openhands.sdk.tool import register_tool, ToolExecutor
from openhands.tools.terminal import TerminalTool
from openhands.tools.file_editor import FileEditorTool
from openhands.tools.task_tracker import TaskTrackerTool

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-4o')}")

# Observability: auto-enabled if LMNR_PROJECT_API_KEY is set
if os.getenv("LMNR_PROJECT_API_KEY"):
    print("✓ Observability: Laminar tracing enabled (view at laminar.sh)")
else:
    print("ℹ Observability: Set LMNR_PROJECT_API_KEY for tracing")

<frozen abc>:106: DeprecationWarning: You should use `Logger` instead. Deprecated since version 1.39.0 and will be removed in a future release.
<frozen abc>:106: DeprecationWarning: You should use `LoggerProvider` instead. Deprecated since version 1.39.0 and will be removed in a future release.
/Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/opentelemetry/_events/__init__.py:201: DeprecationWarning: You should use `ProxyLoggerProvider` instead. Deprecated since version 1.39.0 and will be removed in a future release.
  _PROXY_EVENT_LOGGER_PROVIDER = ProxyEventLoggerProvider()
2026-01-23 21:00:32,591::lmnr.opentelemetry_lib.litellm::WARNING: Laminar LiteLLM callback is deprecated. LiteLLM is already instrumented by Laminar. This callback will not have any effect. You can safely remove the callback from your code. (__init__.py:83)


Model: openai/gpt-5
✓ Observability: Laminar tracing enabled (view at laminar.sh)


## Step 2: Create LLM and Tavily Tool

In [3]:
# Create LLM
llm = LLM(
    model="openai/gpt-4o",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

# Create Tavily client and tool (same as notebook 00)
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

class SearchAction(Action):
    query: str = Field(description="Search query")

class SearchObservation(Observation):
    results: str = Field(description="Results")
    @property
    def to_llm_content(self): return [TextContent(text=self.results)]

class SearchExecutor(ToolExecutor):
    def __call__(self, action: SearchAction, conversation=None) -> SearchObservation:
        try:
            r = tavily.search(query=action.query, max_results=5)
            text = "\n\n".join([
                f"**{x['title']}**\n{x['content'][:300]}\nSource: {x['url']}"
                for x in r['results']
            ])
            return SearchObservation(results=text or "No results found")
        except Exception as e:
            # Error recovery: return error message instead of crashing
            return SearchObservation(results=f"Search failed: {str(e)}. Try a different query.")

class SearchTool(ToolDefinition[SearchAction, SearchObservation]):
    @classmethod
    def create(cls, conv_state) -> List["SearchTool"]:
        return [cls(description="Web search via Tavily (returns results with citations)",
                    action_type=SearchAction, observation_type=SearchObservation, 
                    executor=SearchExecutor())]

register_tool("TavilySearch", SearchTool.create)
print("✓ LLM and Tavily tool ready")

✓ LLM and Tavily tool ready


## Step 3: Create Agent with File Editor

The key difference from notebook 00: we include `FileEditorTool` so the agent can **create and update the research plan file**.

In [4]:
agent = Agent(
    llm=llm,
    tools=[
        Tool(name=FileEditorTool.name),     # Read/write research_plan.md
        Tool(name=TaskTrackerTool.name),    # Track task completion
        Tool(name="TavilySearch"),          # Web search with citations
    ],
)
print("✓ Agent ready with tools:", [t.name for t in agent.tools])

✓ Agent ready with tools: ['file_editor', 'task_tracker', 'TavilySearch']


## Step 4: The Deep Research Prompt

This is where the magic happens. We give the agent explicit instructions to:
1. **Create a plan file** before starting research
2. **Update the plan** as each task completes
3. **Iterate and refine** if initial searches don't yield enough info

In [5]:
DEEP_RESEARCH_PROMPT = """
You are a deep research agent. Your task is to thoroughly research a topic using a structured, iterative approach.

## Your Process:

### Phase 1: Planning
1. Create a file called `research_plan.md` with:
   - The main research question
   - 3-5 specific sub-questions to investigate
   - A checklist of tasks (use markdown checkboxes: `- [ ]` and `- [x]`)

### Phase 2: Research
For each sub-question:
1. Read `research_plan.md` to check what's done and what's next
2. Use TavilySearch to find relevant information
3. Take notes on key findings (you can add them to the plan file)
4. Mark the task as complete in `research_plan.md` (change `- [ ]` to `- [x]`)

### Phase 3: Refinement
After initial research:
1. Review all findings
2. Identify any gaps or areas needing more depth
3. Add new tasks to `research_plan.md` if needed
4. Continue researching until satisfied

### Phase 4: Synthesis
1. Read the final `research_plan.md` 
2. Create a comprehensive summary with:
   - Key findings organized by theme
   - Important sources (URLs)
   - Conclusions and implications

## Rules:
- ALWAYS create and update `research_plan.md` - this is your memory
- ALWAYS include source URLs in your findings (citations)
- If a search fails, try rephrasing the query
- Be thorough but efficient - don't repeat searches

Now research: {topic}
"""

print("✓ Deep research prompt ready")

✓ Deep research prompt ready


## Step 5: Create Critique Agent (GPT-5.1)

Following the [OpenHands iterative refinement pattern](https://docs.openhands.dev/sdk/guides/iterative-refinement), we use a second model to critique and improve the research plan before execution.

In [6]:
# Create a separate LLM for critique (GPT-5.1 for stronger reasoning)
critique_llm = LLM(
    model="openai/gpt-5.1",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

# Critique agent only needs file access (no search needed)
critique_agent = Agent(
    llm=critique_llm,
    tools=[
        Tool(name=FileEditorTool.name),  # Read/write plan and critique files
    ],
)

print(f"✓ Critique agent ready (model: {critique_llm.model})")

✓ Critique agent ready (model: openai/gpt-5.1)


## Step 6: Run the Research Pipeline

The workflow with **checkpointing for resume capability**:

1. **Phase 1 - Planning**: Create plan → Critique → Improve → Loop
2. **Phase 2 - Research**: Execute Tavily searches for each sub-question
3. **Phase 3 - Synthesis**: GPT-5.1 writes comprehensive report

💾 **Checkpointing**: Progress is saved to `checkpoint.json` after each phase. If the notebook crashes or is interrupted, re-run this cell to resume from where you left off.

In [7]:
import re

# Configuration
QUALITY_THRESHOLD = 80.0
MAX_ITERATIONS = 2
cwd = os.getcwd()
CHECKPOINT_FILE = "checkpoint.json"

# Choose your research topic
topic = "What are the latest breakthroughs in AI agents and autonomous systems in 2024-2025?"

# ===========================================
# Checkpoint System for Resume Capability
# ===========================================

def save_checkpoint(phase: str, iteration: int = 0):
    """Save current progress to enable resume after crash/interrupt."""
    checkpoint = {
        "phase": phase,
        "iteration": iteration,
        "topic": topic,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(checkpoint, f, indent=2)
    print(f"  💾 Checkpoint saved: {phase}")

def load_checkpoint():
    """Load checkpoint if exists."""
    try:
        with open(CHECKPOINT_FILE, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return None

def clear_checkpoint():
    """Remove checkpoint after successful completion."""
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)

# Check for existing checkpoint
checkpoint = load_checkpoint()
if checkpoint:
    print(f"🔄 Found checkpoint from {checkpoint['timestamp']}")
    print(f"   Phase: {checkpoint['phase']}, Iteration: {checkpoint.get('iteration', 0)}")
    print(f"   Topic: {checkpoint['topic'][:50]}...")
    resume_from = checkpoint['phase']
else:
    resume_from = None
    print("📝 Starting fresh research (no checkpoint found)")

# Prompts
PLAN_CREATION_PROMPT = """
Create a research plan in `research_plan.md` for the topic: {topic}

The plan should include:
1. Main research question
2. 4-5 specific sub-questions to investigate
3. A checklist of search tasks (use `- [ ]` format)

Just create the plan file - don't execute any searches yet.
"""

CRITIQUE_PROMPT = """
Read `research_plan.md` and evaluate the research plan quality.

Score each dimension (0-25 points):
1. **Comprehensiveness**: Do sub-questions cover all important aspects?
2. **Specificity**: Are questions concrete enough to search effectively?
3. **Diversity**: Do questions explore different angles/perspectives?
4. **Structure**: Is the plan logically organized?

Write your evaluation to `critique.md` with:
- Scores for each dimension
- Specific suggestions for improvement
- Final line MUST be: `SCORE: XX/100`
"""

IMPROVE_PROMPT = """
Read `critique.md` and improve `research_plan.md` based on the feedback.
Address each suggestion while keeping the good parts of the original plan.
"""

def parse_score(filepath):
    """Extract score from critique file."""
    try:
        with open(filepath, "r") as f:
            content = f.read()
        match = re.search(r"SCORE:\s*(\d+)", content, re.IGNORECASE)
        return float(match.group(1)) if match else 0.0
    except FileNotFoundError:
        return 0.0

# ===========================================
# PHASE 1: Planning (with iterative critique)
# ===========================================

research_conversation = Conversation(agent=agent, workspace=cwd)

if resume_from in [None, "planning"]:
    print("=" * 60)
    print("PHASE 1: Creating initial research plan...")
    print("=" * 60)
    
    research_conversation.send_message(PLAN_CREATION_PROMPT.format(topic=topic))
    research_conversation.run()
    save_checkpoint("planning", iteration=0)
    
    # Iterative refinement loop
    start_iteration = checkpoint.get('iteration', 0) if resume_from == "planning" else 0
    for iteration in range(start_iteration, MAX_ITERATIONS):
        print(f"\n{'=' * 60}")
        print(f"ITERATION {iteration + 1}: Critique agent evaluating plan...")
        print("=" * 60)
        
        critique_conversation = Conversation(agent=critique_agent, workspace=cwd)
        critique_conversation.send_message(CRITIQUE_PROMPT)
        critique_conversation.run()
        
        score = parse_score("critique.md")
        print(f"\n→ Plan quality score: {score}/100")
        
        if score >= QUALITY_THRESHOLD:
            print(f"✓ Plan approved! (threshold: {QUALITY_THRESHOLD})")
            break
        else:
            print(f"✗ Below threshold ({QUALITY_THRESHOLD}). Improving...")
            research_conversation.send_message(IMPROVE_PROMPT)
            research_conversation.run()
            save_checkpoint("planning", iteration=iteration+1)
else:
    print("⏭️  Skipping Phase 1 (already complete)")

# ===========================================
# PHASE 2: Research (execute searches)
# ===========================================

if resume_from in [None, "planning", "research"]:
    print(f"\n{'=' * 60}")
    print("PHASE 2: Executing research plan...")
    print("=" * 60)
    
    EXECUTE_PROMPT = """
    Read `research_plan.md` and execute each task:

    1. For each unchecked task (`- [ ]`):
       - Use TavilySearch to find information
       - Add the RAW findings (with source URLs) under each task
       - Mark the task complete (`- [x]`)

    Keep all findings in `research_plan.md` - this is your research notes.
    Do NOT write a summary yet - just gather all the information.
    """
    
    research_conversation.send_message(EXECUTE_PROMPT)
    research_conversation.run()
    save_checkpoint("research")
    
    print("\n✓ Research gathering complete!")
else:
    print("⏭️  Skipping Phase 2 (already complete)")

📝 Starting fresh research (no checkpoint found)


[01/23/26 21:00:32] INFO     Created new conversation 04538ed5-ea51-4b29-bc7f-84effdd2f97b             ]8;id=457221;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=368225;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('04538ed5-ea51-4b29-bc7f-84effdd2f97b'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-4o', 'api_key':                                  
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 128000,                         
                             'max_output_tokens': 16384, 'stream': False, 'drop_params': True,                     
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'file_editor', 'params':                
                             {}}, {'name': 'task_tracker', 'params': {}}, {'name': 'TavilySearch',                 
                             'params': {}}], 'mcp_config': {}, 'include_default_tools': ['FinishTool',             
                             'ThinkTool'], 'system_prompt_filename': 'system_prompt.j2',                           
                             'security_policy_filename': 'security_policy.j2', 'system_prompt_kwargs':             
                             {'llm_security_analyzer': True}, 'kind': 'Agent'}                                     

[01/23/26 21:00:32] INFO     TaskTrackerExecutor initialized with save_dir: None                  ]8;id=979178;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/task_tracker/definition.py\definition.py]8;;\:]8;id=317076;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/task_tracker/definition.py#161\161]8;;\

[01/23/26 21:00:32] INFO     FileEditor initialized with cwd:                                          ]8;id=787466;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py\editor.py]8;;\:]8;id=309154;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py#95\95]8;;\
                             /Users/rajivshah/Code/openhands-deep-agent                                            

[01/23/26 21:00:32] INFO     Loaded 3 tools from spec: ['file_editor', 'task_tracker', 'search']        ]8;id=459428;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py\base.py]8;;\:]8;id=105080;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py#281\281]8;;\

...

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
[File /Users/rajivshah/Code/openhands-deep-agent/research_plan.md edited with 1 changes.]
[begin of edit 1 / 1]
(content before edit)
 64|    - [Opportunities, Risks, and Legal Challenges for AI Agents - Galkin 
Law](https://galkinlaw.com/2025-the-year-of-ai-agents/): Explores the opportunities, risks, and legal challenges 
for use of AI agents in business.
 65|    - [What are the risks and benefits of 'AI agents'? | World Economic 
Forum](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/): Discusses ethical 
questions about decision-making and accountability, as well as socioeconomic risks.
-66|- [ ] Compare open-source and proprietary ecosystems in AI innovation (Q13)
 67|- [ ] Investigate geographical or sectoral differences in AI deployment and regulation (Q14)
 68|- [ ] Summarize cross-cutting themes and tensions across technical, application, and governance findings    - 
[The Future of AI Agents: Transforming Business and Society in 
2025](https://www.linkedin.com/pulse/future-ai-agents-transforming-business-society-2025-beyond-john-enoh-hjo6c): 
AI agents are significantly transforming healthcare and scientific discovery.
(content after edit)
 64|    - [Opportunities, Risks, and Legal Challenges for AI Agents - Galkin 
Law](https://galkinlaw.com/2025-the-year-of-ai-agents/): Explores the opportunities, risks, and legal challenges 
for use of AI agents in business.
 65|    - [What are the risks and benefits of 'AI agents'? | World Economic 
Forum](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/): Discusses ethical 
questions about decision-making and accountability, as well as socioeconomic risks.
+66|- [x] Compare open-source and proprietary ecosystems in AI innovation (Q13)
+67|  - **Findings**:
+68|    - [Best Open-Source AI Platforms for 2025: The Frameworks Powering Next-Gen ML and 
LLMs](https://greennode.ai/blog/best-open-source-ai-platforms): Open-source AI platforms are reshaping how machine 
learning and large language models are built, deployed, and scaled.
+69|    - [Open Source vs Proprietary AI Models: Who's Winning the Race in 
2025?](https://medium.com/@simplenight/open-source-vs-proprietary-ai-models-whos-winning-the-race-in-2025-1370ef81e
4bc): 2025 has become a battleground between open source models and proprietary AI models.
+70|    - [Open Source AI is Transforming the Economy—Here's What the Linux Foundation 
Says](https://www.linuxfoundation.org/blog/open-source-ai-is-transforming-the-economy): Describes the nuances of 
how open source AI is impacting the global economy and workforce.
+71|    - [Open-Source Software versus Proprietary Software in 
2025](https://www.planetcrust.com/open-source-software-v-proprietary-software-2025/?utm_campaign=blog): Open-source
software is beginning to assert its dominance with greater flexibility and quality.
+72|    - [Open Source AI vs Proprietary AI: Strategic Guide for 
Enterprises](https://codeninjaconsulting.com/blog/open-source-ai-vs-proprietary-ai-infrastructure-for-enterprise-AI
): The proliferation of open source AI capabilities has fundamentally altered the strategic calculus of open source
AI vs proprietary AI infrastructure.
 73|- [ ] Investigate geographical or sectoral differences in AI deployment and regulation (Q14)
 74|- [ ] Summarize cross-cutting themes and tensions across technical, application, and governance findings    - 
[The Future of AI Agents: Transforming Business and Society in 
2025](https://www.linkedin.com/pulse/future-ai-agents-transforming-business-society-2025-beyond-john-enoh-hjo6c): 
AI agents are significantly transforming healthcare and scientific discovery.
[end of edit 1 / 1]

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Summary: file_editor: {"command": "str_replace", "path": 
"/Users/rajivshah/Code/openhands-deep-agent/research_plan.md", "old_str": "- [ ] Investigate geographical or 
sectoral differences in AI deployment and regulation (Q14)", "new_str": "- [x] Investigate geographical or sectoral
differences in AI deployment and regulation (Q14)\n  - **Findings**:\n    - [The Updated State of AI Regulations 
for 2025 - Cimplifi](https://www.cimplifi.com/resources/the-updated-state-of-ai-regulations-for-2025/): Explores 
how 2025 global AI regulations are reshaping legal compliance, risk, and strategy.\n    - [Emerging divides in the 
transition to artificial intelligence - 
OECD](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-a
rtificial-intelligence_eeb5e120/7376c776-en.pdf): Gaps in AI adoption rates have widened across places, sectors, 
and firms.\n    - [Blind Spots in AI Governance: Military AI and the EU's Regulatory Oversight 
Gap](https://esthinktank.com/2025/10/03/blind-spots-in-ai-governance-military-ai-and-the-eus-regulatory-oversight-g
ap/): Discusses the deployment of AI without geographical or political boundaries.\n    - [AI Regulation by 
Geography: How Jurisdictional Differences Have 
Emerged](https://www.garp.org/risk-intelligence/culture-governance/ai-regulation-geography-240712): Regulators and 
governments are aligning on how to gauge, design, and implement AI regulations.\n    - [The EU AI Act: What 
Businesses Need To Know - Skadden 
Arps](https://www.skadden.com/insights/publications/2024/06/quarterly-insights/the-eu-ai-act-what-businesses-need-t
o-know): Defines AI systems broadly and outlines obligations for providers, deployers, importers, and 
distributors."}

Action: FileEditorAction

Arguments:
  command: "str_replace"
  path: "/Users/rajivshah/Code/openhands-deep-agent/research_plan.md"
  old_str: "- [ ] Investigate geographical or sectoral differences in AI deployment and regulation (Q14)"
  new_str: 
    - [x] Investigate geographical or sectoral differences in AI deployment and regulation (Q14)
      - **Findings**:
        - [The Updated State of AI Regulations for 2025 - 
Cimplifi](https://www.cimplifi.com/resources/the-updated-state-of-ai-regulations-for-2025/): Explores how 2025 
global AI regulations are reshaping legal compliance, risk, and strategy.
        - [Emerging divides in the transition to artificial intelligence - 
OECD](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-a
rtificial-intelligence_eeb5e120/7376c776-en.pdf): Gaps in AI adoption rates have widened across places, sectors, 
and firms.
        - [Blind Spots in AI Governance: Military AI and the EU's Regulatory Oversight 
Gap](https://esthinktank.com/2025/10/03/blind-spots-in-ai-governance-military-ai-and-the-eus-regulatory-oversight-g
ap/): Discusses the deployment of AI without geographical or political boundaries.
        - [AI Regulation by Geography: How Jurisdictional Differences Have 
Emerged](https://www.garp.org/risk-intelligence/culture-governance/ai-regulation-geography-240712): Regulators and 
governments are aligning on how to gauge, design, and implement AI regulations.
        - [The EU AI Act: What Businesses Need To Know - Skadden 
Arps](https://www.skadden.com/insights/publications/2024/06/quarterly-insights/the-eu-ai-act-what-businesses-need-t
o-know): Defines AI systems broadly and outlines obligations for providers, deployers, importers, and distributors.

  kind: "FileEditorAction"

Tokens: ↑ input 548.5K • cache hit 84.03% • ↓ output 9.33K • $ 0.8884

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
[File /Users/rajivshah/Code/openhands-deep-agent/research_plan.md edited with 1 changes.]
[begin of edit 1 / 1]
(content before edit)
 71|    - [Open-Source Software versus Proprietary Software in 
2025](https://www.planetcrust.com/open-source-software-v-proprietary-software-2025/?utm_campaign=blog): Open-source
software is beginning to assert its dominance with greater flexibility and quality.
 72|    - [Open Source AI vs Proprietary AI: Strategic Guide for 
Enterprises](https://codeninjaconsulting.com/blog/open-source-ai-vs-proprietary-ai-infrastructure-for-enterprise-AI
): The proliferation of open source AI capabilities has fundamentally altered the strategic calculus of open source
AI vs proprietary AI infrastructure.
-73|- [ ] Investigate geographical or sectoral differences in AI deployment and regulation (Q14)
 74|- [ ] Summarize cross-cutting themes and tensions across technical, application, and governance findings    - 
[The Future of AI Agents: Transforming Business and Society in 
2025](https://www.linkedin.com/pulse/future-ai-agents-transforming-business-society-2025-beyond-john-enoh-hjo6c): 
AI agents are significantly transforming healthcare and scientific discovery.
 75|    - [IQVIA launches new AI agents for life sciences and 
healthcare](https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare): 
Custom-built AI agents enhance workflows and accelerate insights in life sciences.
(content after edit)
 71|    - [Open-Source Software versus Proprietary Software in 
2025](https://www.planetcrust.com/open-source-software-v-proprietary-software-2025/?utm_campaign=blog): Open-source
software is beginning to assert its dominance with greater flexibility and quality.
 72|    - [Open Source AI vs Proprietary AI: Strategic Guide for 
Enterprises](https://codeninjaconsulting.com/blog/open-source-ai-vs-proprietary-ai-infrastructure-for-enterprise-AI
): The proliferation of open source AI capabilities has fundamentally altered the strategic calculus of open source
AI vs proprietary AI infrastructure.
+73|- [x] Investigate geographical or sectoral differences in AI deployment and regulation (Q14)
+74|  - **Findings**:
+75|    - [The Updated State of AI Regulations for 2025 - 
Cimplifi](https://www.cimplifi.com/resources/the-updated-state-of-ai-regulations-for-2025/): Explores how 2025 
global AI regulations are reshaping legal compliance, risk, and strategy.
+76|    - [Emerging divides in the transition to artificial intelligence - 
OECD](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-a
rtificial-intelligence_eeb5e120/7376c776-en.pdf): Gaps in AI adoption rates have widened across places, sectors, 
and firms.
+77|    - [Blind Spots in AI Governance: Military AI and the EU's Regulatory Oversight 
Gap](https://esthinktank.com/2025/10/03/blind-spots-in-ai-governance-military-ai-and-the-eus-regulatory-oversight-g
ap/): Discusses the deployment of AI without geographical or political boundaries.
+78|    - [AI Regulation by Geography: How Jurisdictional Differences Have 
Emerged](https://www.garp.org/risk-intelligence/culture-governance/ai-regulation-geography-240712): Regulators and 
governments are aligning on how to gauge, design, and implement AI regulations.
+79|    - [The EU AI Act: What Businesses Need To Know - Skadden 
Arps](https://www.skadden.com/insights/publications/2024/06/quarterly-insights/the-eu-ai-act-what-businesses-need-t
o-know): Defines AI systems broadly and outlines obligations for providers, deployers, importers, and distributors.
 80|- [ ] Summarize cross-cutting themes and tensions across technical, application, and governance findings    - 
[The Future of AI Agents: Transforming Business and Society in 
2025](https://www.linkedin.com/pulse/future-ai-agents-tr

Message from Agent ────────────────────────────────────────────────────────────────────────────────────────────────

All tasks in the `research_plan.md` have been executed, and the findings have been added under each task. Here is a
summary of the actions taken:

1. **Identified key publications and papers on AI agent architectures from 2024-2025**: Added findings from various
sources discussing advancements in AI agent architectures.

2. **Explored advancements in decision-making and adaptability of autonomous systems**: Included findings on new 
generation agents, manufacturing evolution, and decision-making systems.

3. **Investigated new benchmarks and evaluation methods for AI agents**: Added insights on evaluation benchmarks 
and methodologies.

4. **Reviewed enabling infrastructure and tooling that supported breakthroughs**: Documented findings on AI 
infrastructure, tools, and platforms.

5. **Analyzed challenges and limitations reported in recent studies on AI agents and autonomous systems**: Included
challenges related to AI adoption, ethical considerations, and legal challenges.

6. **Compared open-source and proprietary ecosystems in AI innovation**: Added findings on the impact and strategic
considerations of open-source vs proprietary AI models.

7. **Investigated geographical or sectoral differences in AI deployment and regulation**: Documented differences in
AI regulations and adoption across regions and sectors.

If you need further analysis or a summary of these findings, feel free to let me know!

Tokens: ↑ input 589.53K • cache hit 81.03% • ↓ output 9.59K • $ 0.9726

  💾 Checkpoint saved: research

✓ Research gathering complete!


In [8]:
# ===========================================
# PHASE 3: Synthesis (comprehensive report)
# ===========================================

print(f"\n{'=' * 60}")
print("PHASE 3: Synthesizing comprehensive report (GPT-5.1)...")
print("=" * 60)

SYNTHESIS_PROMPT = """
You are a research synthesis expert. Read `research_plan.md` which contains:
- The original research question
- Sub-questions that were investigated
- Raw findings from web searches with source URLs

Your task: Write a comprehensive research report to `research_report.md`

## Report Structure:

# [Research Topic Title]

## Executive Summary
(2-3 paragraph overview of key findings and conclusions)

## 1. Introduction
(Context, why this topic matters, scope of research)

## 2. Key Findings

### 2.1 [Theme 1 from the research]
(Synthesize related findings into coherent paragraphs, not bullet points)
(Cite sources inline like: "According to [Source Name](url)...")

### 2.2 [Theme 2]
(Continue for each major theme discovered)

### 2.3 [Theme 3]
...

## 3. Analysis & Implications
(What do these findings mean? Trends, patterns, implications for the future)

## 4. Limitations
(What wasn't covered? What uncertainties remain?)

## 5. Conclusion
(Final synthesis and key takeaways)

## References
(List all sources with URLs)

---

IMPORTANT:
- Write in a professional, analytical tone
- Synthesize and connect ideas - don't just list facts
- Every claim should reference a source
- Be comprehensive but concise
- Make it feel like a real research report, not a summary of search results
"""

# Use the critique agent (GPT-5.1) for synthesis - it has stronger reasoning
synthesis_conversation = Conversation(agent=critique_agent, workspace=cwd)
synthesis_conversation.send_message(SYNTHESIS_PROMPT)
synthesis_conversation.run()

# Clear checkpoint on successful completion
clear_checkpoint()
print("\n✓ Research report complete!")
print("✓ Checkpoint cleared (research finished successfully)")


PHASE 3: Synthesizing comprehensive report (GPT-5.1)...


[01/23/26 21:08:10] INFO     Created new conversation 01a7263e-8e4c-4e9b-bc3f-6930aa4b6ab1             ]8;id=6765;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=851992;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('01a7263e-8e4c-4e9b-bc3f-6930aa4b6ab1'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-5.1', 'api_key':                                 
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 272000,                         
                             'max_output_tokens': 128000, 'stream': False, 'drop_params': True,                    
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'file_editor', 'params':                
                             {}}], 'mcp_config': {}, 'include_default_tools': ['FinishTool',                       
                             'ThinkTool'], 'system_prompt_filename': 'system_prompt.j2',                           
                             'security_policy_filename': 'security_policy.j2', 'system_prompt_kwargs':             
                             {'llm_security_analyzer': True}, 'kind': 'Agent'}                                     

[01/23/26 21:08:10] WARNING  Agent already initialized; skipping re-initialization.                     ]8;id=528162;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py\base.py]8;;\:]8;id=24163;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py#257\257]8;;\

System Prompt ─────────────────────────────────────────────────────────────────────────────────────────────────────

System Prompt:
You are OpenHands agent, a helpful AI assistant that can interact with a computer to solve tasks.

<ROLE>
* Your primary role is to assist users by executing commands, modifying code, and solving technical problems 
effectively. You should be thorough, methodical, and prioritize quality over speed.
* If the user asks a question, like "why is X happening", don't try to fix the problem. Just give an answer to the 
question.
</ROLE>

<MEMORY>
* Use `AGENTS.md` under the repository root as your persistent memory for repository-specific knowledge and 
context.
* Add important insights, patterns, and learnings to this file to improve future task performance.
* This repository skill is automatically loaded for every conversation and helps maintain context across sessions.
* For more information about skills, see: https://docs.openhands.dev/overview/skills
</MEMORY>

<EFFICIENCY>
* Each action you take is somewhat expensive. Wherever possible, combine multiple actions into a single action, 
e.g. combine multiple bash commands into one, using sed and grep to edit/view multiple files at once.
* When exploring the codebase, use efficient tools like find, grep, and git commands with appropriate filters to 
minimize unnecessary operations.
</EFFICIENCY>

<FILE_SYSTEM_GUIDELINES>
* When a user provides a file path, do NOT assume it's relative to the current working directory. First explore the
file system to locate the file before working on it.
* If asked to edit a file, edit the file directly, rather than creating a new file with a different filename.
* For global search-and-replace operations, consider using `sed` instead of opening file editors multiple times.
* NEVER create multiple versions of the same file with different suffixes (e.g., file_test.py, file_fix.py, 
file_simple.py). Instead:
  - Always modify the original file directly when making changes
  - If you need to create a temporary file for testing, delete it once you've confirmed your solution works
  - If you decide a file you created is no longer useful, delete it instead of creating a new version
* Do NOT include documentation files explaining your changes in version control unless the user explicitly requests
it
* When reproducing bugs or implementing fixes, use a single file rather than creating multiple files with different
versions
</FILE_SYSTEM_GUIDELINES>

<CODE_QUALITY>
* Write clean, efficient code with minimal comments. Avoid redundancy in comments: Do not repeat information that 
can be easily inferred from the code itself.
* When implementing solutions, focus on making the minimal changes needed to solve the problem.
* Before implementing any changes, first thoroughly understand the codebase through exploration.
* If you are adding a lot of code to a function or file, consider splitting the function or file into smaller 
pieces when appropriate.
* Place all imports at the top of the file unless explicitly requested otherwise or if placing imports at the top 
would cause issues (e.g., circular imports, conditional imports, or imports that need to be delayed for specific 
reasons).
</CODE_QUALITY>

<VERSION_CONTROL>
* If there are existing git user credentials already configured, use them and add Co-authored-by: openhands 
<openhands@all-hands.dev> to any commits messages you make. if a git config doesn't exist use "openhands" as the 
user.name and "openhands@all-hands.dev" as the user.email by default, unless explicitly instructed otherwise.
* Exercise caution with git operations. Do NOT make potentially dangerous changes (e.g., pushing to main, deleting 
repositories) unless explicitly asked to do so.
* When committing changes, use `git status` to see all modified files, and stage all files necessary for the 
commit. Use `git commit -a` whenever possible.
* Do NOT commit files that typically shouldn't go into version control (e.g.

Message from User ─────────────────────────────────────────────────────────────────────────────────────────────────


You are a research synthesis expert. Read `research_plan.md` which contains:
- The original research question
- Sub-questions that were investigated
- Raw findings from web searches with source URLs

Your task: Write a comprehensive research report to `research_report.md`

## Report Structure:

# [Research Topic Title]

## Executive Summary
(2-3 paragraph overview of key findings and conclusions)

## 1. Introduction
(Context, why this topic matters, scope of research)

## 2. Key Findings

### 2.1 [Theme 1 from the research]
(Synthesize related findings into coherent paragraphs, not bullet points)
(Cite sources inline like: "According to [Source Name](url)...")

### 2.2 [Theme 2]
(Continue for each major theme discovered)

### 2.3 [Theme 3]
...

## 3. Analysis & Implications
(What do these findings mean? Trends, patterns, implications for the future)

## 4. Limitations
(What wasn't covered? What uncertainties remain?)

## 5. Conclusion
(Final synthesis and key takeaways)

## References
(List all sources with URLs)

---

IMPORTANT:
- Write in a professional, analytical tone
- Synthesize and connect ideas - don't just list facts
- Every claim should reference a source
- Be comprehensive but concise
- Make it feel like a real research report, not a summary of search results


Tokens: ↑ input 164.7K • cache hit 77.10% •  reasoning 10.94K • ↓ output 15.17K • $ 0.2147

...

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
NOTE: We only show up to 172 since there're only 172 lines in this file.
Here's the result of running `cat -n` on /Users/rajivshah/Code/openhands-deep-agent/research_report.md:
   142  - "Human-AI Interaction Strategies" – Emergent Mind 
(https://www.emergentmind.com/topics/human-ai-interaction-strategies)
   143  - "Societal Implications of AI: Key Insights from Emerging Signals" – UNESCO 
(https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight
%20from%20Signals.pdf)
   144  - "AI at Work 2025: Momentum Builds, but Gaps Remain" – BCG 
(https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain)
   145  - "Is AI Closing the Door on Entry-Level Job Opportunities?" – World Economic Forum 
(https://www.weforum.org/stories/2025/04/ai-jobs-international-workers-day/)
   146  - "AI and the Future of Work" – National Fund for Workforce Solutions 
(https://nationalfund.org/ai-and-the-future-of-work/)
   147  - "AI and the Future of Work – TIAA" – TIAA Institute 
(https://www.tiaa.org/content/dam/tiaa/institute/pdf/insights-report/2025-10/tiaa-institute-insights-brief-ai-and-t
he-future-of-work-reshaping-the-landscape-of-human-work-watson-mastry-aug-2025.pdf)
   148  - "AI and Privacy: Shifting from 2024 to 2025" – Cloud Security Alliance 
(https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal
-developments)
   149  - "Regulating Artificial Intelligence: U.S. and International Approaches" – CRS 
(https://www.congress.gov/crs-product/R48555)
   150  - "Responsible AI: Ethical Policies and Practices" – Microsoft 
(https://www.microsoft.com/en-us/ai/responsible-ai)
   151  - "AI Compliance in 2025: Global Regulations, Risks & Best Practices" – Vodworks 
(https://vodworks.com/blogs/ai-compliance/)
   152  - "Data Privacy & AI Ethics Best Practices" – TrustCloud 
(https://community.trustcloud.ai/docs/grc-launchpad/grc-101/governance/data-privacy-and-ai-ethical-considerations-a
nd-best-practices/)
   153  - "What Are the Risks and Benefits of 'AI Agents'?" – World Economic Forum 
(https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/)
   154  - "Fully Autonomous AI Agents Should Not be Developed" – arXiv (https://arxiv.org/html/2502.02649v3)
   155  - "Adoption of AI and Agentic Systems: Value, Challenges, and Pathways" – California Management Review 
(https://cmr.berkeley.edu/2025/08/adoption-of-ai-and-agentic-systems-value-challenges-and-pathways/)
   156  - "Safety Challenges for Generative AI Agents in Autonomous Machines" – Medium 
(https://cobusgreyling.medium.com/safety-challenges-for-generative-ai-agents-in-autonomous-machines-081846ea6753)
   157  - "AI Agents vs. Agentic AI: A Conceptual Taxonomy" – Journal article 
(https://www.sciencedirect.com/science/article/pii/S1566253525006712)
   158  - "Open Source vs Proprietary AI Models: Who's Winning the Race in 2025?" – Medium 
(https://medium.com/@simplenight/open-source-vs-proprietary-ai-models-whos-winning-the-race-in-2025-1370ef81e4bc)
   159  - "Open Source Technology in the Age of AI" – McKinsey 
(https://www.mckinsey.com/~/media/mckinsey/business%20functions/quantumblack/our%20insights/open%20source%20technol
ogy%20in%20the%20age%20of%20ai/open-source-technology-in-the-age-of-ai_final.pdf)
   160  - "Open-Source Software versus Proprietary Software in 2025" – PlanetCrust 
(https://www.planetcrust.com/open-source-software-v-proprietary-software-2025/?utm_campaign=blog)
   161  - "Best Open-Source AI Platforms for 2025" – Greennode 
(https://greennode.ai/blog/best-open-source-ai-platforms)
   162  - "Open-source AI: Are Younger Developers Leading the Way?" – Stack Overflow Blog 
(https://stackoverflow.blog/2025/04/07/open-source-ai-are-younger-developers-leading-the-way/)


Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: View section 3 Analysis & Implications to ensure completeness and citations

Reasoning:
Action: FileEditorAction

Arguments:
  command: "view"
  path: "/Users/rajivshah/Code/openhands-deep-agent/research_report.md"
  view_range: [60, 100]
  kind: "FileEditorAction"

Tokens: ↑ input 350.95K • cache hit 81.33% •  reasoning 16.45K • ↓ output 21.34K • $ 0.3310

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: file_editor
Result:
Here's the result of running `cat -n` on /Users/rajivshah/Code/openhands-deep-agent/research_report.md:
    60  
    61  Studies of human‑AI collaboration find that, despite the rhetoric of “co‑pilots,” many current systems 
function more as advanced decision‑support tools than as truly collaborative partners, with humans often adapting 
their workflows to AI system affordances rather than vice versa ([Human‑AI collaboration 
taxonomy](https://www.frontiersin.org/journals/computer-science/articles/10.3389/fcomp.2024.1521066/full); 
[Taxonomy of interaction 
patterns](https://pure.johnshopkins.edu/en/publications/human-ai-collaboration-is-not-very-collaborative-yet-a-taxo
nomy-o); [Human‑AI interaction in creative domains](https://dl.acm.org/doi/10.1145/3743049.3743081)). Additional 
work on human‑AI co‑creation and interaction strategies highlights the importance of control mechanisms, 
transparency, and context‑sensitive interfaces to support trust and effective collaboration in creative, 
analytical, and operational tasks ([Exploring collaboration patterns in human‑AI 
co‑creation](https://arxiv.org/pdf/2507.06000); [Human‑AI interaction 
strategies](https://www.emergentmind.com/topics/human-ai-interaction-strategies)).
    62  
    63  ### 2.6 Governance, Risks, and Geographical Differences
    64  
    65  Regulatory and governance frameworks for AI agents are evolving rapidly, with a notable shift toward 
risk‑based approaches that classify systems by potential harm and impose graduated requirements for transparency, 
data governance, and oversight. Analyses of global regulatory trends describe how the EU’s AI Act is shaping 
international discourse through its focus on high‑risk use cases, conformity assessments, and post‑market 
monitoring, influencing firms far beyond Europe’s borders ([Artificial Intelligence 2025 – Chambers 
guide](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [Dentons on AI regulation
and 
ethics](https://www.dentons.com/en/insights/articles/2025/january/10/ai-trends-for-2025-ai-regulation-governance-an
d-ethics); [AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/)). U.S. and international overviews 
highlight emerging legislation and guidance that address algorithmic discrimination, consumer protection, and 
safety, including state laws such as Colorado’s AI statute, as well as sector‑specific rules in finance and 
healthcare ([CRS report on regulating AI](https://www.congress.gov/crs-product/R48555); [GARP on AI regulation by 
geography](https://www.garp.org/risk-intelligence/culture-governance/ai-regulation-geography-240712); [Cloud 
Security Alliance on AI and 
privacy](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-glob
al-legal-developments)).
    66  
    67  Corporate responsible‑AI frameworks complement formal regulations by providing principles and operational 
guidance on fairness, transparency, privacy, and security in AI tools and agents, often codified in internal 
policies, tooling, and review processes ([Microsoft Responsible 
AI](https://www.microsoft.com/en-us/ai/responsible-ai); [TrustCloud governance 
guidance](https://community.trustcloud.ai/docs/grc-launchpad/grc-101/governance/data-privacy-and-ai-ethical-conside
rations-and-best-practices/); [Medium on AI ethics in 
2025](https://medium.com/@mmp3071/ai-ethics-in-2025-balancing-innovation-with-responsibility-4a267f1b7717)). 
Discussions of AI compliance in 2025 emphasize requirements for bias testing, documentation, and monitoring in 
high‑risk systems, including many agentic and autonomous applications in finance, employment, and critical 
infrastructure ([AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/); [Cloud Security Alliance on AI 
and 
privacy](https://clouds

Message from Agent ────────────────────────────────────────────────────────────────────────────────────────────────

`research_report.md` already contains a full report matching your requested structure and grounded in the sources 
from `research_plan.md`. Here is what is now in the file:

# Breakthroughs in AI Agents and Autonomous Systems (2024–2025)

## Executive Summary  
Three paragraphs synthesize:

- How AI agents and autonomous systems shifted from proof‑of‑concepts to infrastructure‑level components, driven by
advances in agentic LLM architectures, autonomous decision‑making, and AI infrastructure and hardware ([Springer 
survey on agentic AI architectures](https://link.springer.com/article/10.1007/s10462-025-11422-4); [Top AI agent 
research round‑ups](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [AI Infrastructure 
2025](https://enkiai.com/ai-market-intelligence/ai-infrastructure-2025-master-your-cloud-gpu-strategy); [Deloitte 
on AI 
hardware](https://www.deloitte.com/us/en/insights/focus/tech-trends/2025/tech-trends-ai-hardware-and-computation-le
ading-ai-revolution.html)).  
- Sectoral deployment and impact in healthcare, life sciences, finance, and operations, with documented 
productivity and workflow gains but also uneven performance and integration challenges ([xCube Labs real‑world 
agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/); [IQVIA healthcare 
agents](https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare); 
[Vouched on real‑life 
agents](https://www.vouched.id/learn/best-of-2025-real-life-ai-agents-are-at-work-in-these-fields); [Datagrid agent
statistics](https://www.datagrid.com/blog/ai-agent-statistics)).  
- The rapidly evolving but fragmented governance landscape, focusing on risk‑based regulation, transparency, and 
safety, alongside evidence that fully autonomous, open‑ended agents remain risky under current conditions ([Cloud 
Security 
Alliance](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-glo
bal-legal-developments); [Chambers AI 2025 
guide](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [World Economic Forum on 
AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/); [“Fully 
Autonomous AI Agents Should Not be Developed”](https://arxiv.org/html/2502.02649v3)).

## 1. Introduction  
Sets context and scope:

- Defines AI agents and autonomous systems (software‑only LLM agents, multi‑agent systems, and embodied robots) and
explains the shift from static prompting to architectures with planning, tool use, memory, and feedback ([Springer 
survey](https://link.springer.com/article/10.1007/s10462-025-11422-4); [Google Cloud on agentic 
apps](https://medium.com/google-cloud/getting-ready-for-a-wave-of-agentic-apps-in-2025-check-out-these-articles-89b
605c41e54)).  
- Describes parallel advances in autonomous systems for manufacturing, mobility, and aerospace, emphasizing robust 
real‑time decision‑making under uncertainty ([MIT AeroAstro autonomous 
systems](https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/); [Comprehensive review of 
autonomous vehicle 
decision‑making](http://www.coscipress.com/api/file/download/20251118/1716055e0587110-5dc1-4cdb-b3e9-dea19f86486d/E
volution%20of%20Decision-making%20Systems%20in%20Autonomous%20Vehicle%20Technology%20-%20A%20Comprehensive%20Review
.pdf)).  
- Frames the report around three pillars from the plan: technical advances; applications and impact; governance and
risks, explicitly tying each to the source clusters in `research_plan.md` ([Analytics Vidhya paper 
round‑up](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [McKinsey on open‑source in 
AI](https://www.mckinsey.com/~/media/mckinsey/business%20functions/quantumblack/our%20insights/open%20source%20tech



✓ Research report complete!
✓ Checkpoint cleared (research finished successfully)


## Step 7: View the Comprehensive Research Report

GPT-5.1 synthesized all findings into a professional `research_report.md`:

In [9]:
from IPython.display import Markdown, display

# Show the comprehensive research report (synthesized by GPT-5.1)
try:
    with open("research_report.md", "r") as f:
        display(Markdown(f.read()))
except FileNotFoundError:
    print("research_report.md not found - run the cells above first")

# Breakthroughs in AI Agents and Autonomous Systems (2024–2025)

## Executive Summary

Between 2024 and 2025, AI agents and autonomous systems moved from proof‑of‑concept deployments toward increasingly central roles in digital infrastructure, enabled by rapid advances in LLM‑based agent architectures, autonomous decision‑making, and specialized compute and cloud platforms ([Springer survey on agentic AI architectures](https://link.springer.com/article/10.1007/s10462-025-11422-4); [Top AI agent research overviews](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [Most influential arXiv AI papers](https://www.paperdigest.org/2025/03/most-influential-arxiv-artificial-intelligence-papers-2025-03-version/); [AI Infrastructure 2025](https://enkiai.com/ai-market-intelligence/ai-infrastructure-2025-master-your-cloud-gpu-strategy); [Deloitte on AI hardware](https://www.deloitte.com/us/en/insights/focus/tech-trends/2025/tech-trends-ai-hardware-and-computation-leading-ai-revolution.html)). Research and industry reports describe a shift from single, prompt‑driven models toward multi‑tool, multi‑agent systems capable of long‑horizon planning, web and tool use, and embodied interaction in robotics and cyber‑physical systems ([Google Cloud on agentic apps](https://medium.com/google-cloud/getting-ready-for-a-wave-of-agentic-apps-in-2025-check-out-these-articles-89b605c41e54); [MIT AeroAstro on autonomous systems](https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/); [Evolution of autonomous systems](https://www.researchgate.net/publication/386099540_The_Evolution_of_AI_in_Autonomous_Systems_Innovations_Challenges_and_Future_Prospects)).

Deployments accelerated across sectors such as healthcare, life sciences, finance, operations, and software engineering, with empirical case studies documenting productivity gains and workflow transformation but also uneven performance and integration challenges ([xCube Labs real‑world agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/); [IQVIA healthcare agents](https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare); [Vouched on real‑life AI agents](https://www.vouched.id/learn/best-of-2025-real-life-ai-agents-are-at-work-in-these-fields); [IBM expectations vs reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality); [Datagrid agent statistics](https://www.datagrid.com/blog/ai-agent-statistics)). Macroeconomic analyses project that AI and emerging technologies will materially reshape growth, sectoral productivity, and global competitiveness, while labor market studies highlight both new high‑skill roles and significant concerns about displacement and job quality ([ETA economic impact report](https://www.joineta.org/blog/economic-impact-and-projections-report-the-integration-of-ai-and-emerging-technologies-2025-2030); [KPMG on generative AI and growth](https://kpmg.com/kpmg-us/content/dam/kpmg/pdf/2025/gen-ai-economic-growth.pdf); [IMF on the global impact of AI](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf); [BCG AI at Work 2025](https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain); [UNESCO societal implications of AI](https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight%20from%20Signals.pdf)).

Governance, safety, and ethics frameworks are evolving but remain fragmented across jurisdictions, with a strong trend toward risk‑based regulation, transparency requirements, and mandatory testing for high‑risk systems ([Cloud Security Alliance on AI and privacy](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal-developments); [EU AI Act overviews](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [CS Congress report on AI regulation](https://www.congress.gov/crs-product/R48555); [Vodworks on AI compliance](https://vodworks.com/blogs/ai-compliance/)). At the same time, red‑teaming studies and conceptual taxonomies underscore unresolved safety issues—including misaligned objectives, lack of formal guarantees, and catastrophic failures in autonomous settings—leading some researchers to argue against fully autonomous open‑ended agents under current conditions ([World Economic Forum on AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/); ["Fully Autonomous AI Agents Should Not be Developed"](https://arxiv.org/html/2502.02649v3); [Safety challenges for generative AI agents](https://cobusgreyling.medium.com/safety-challenges-for-generative-ai-agents-in-autonomous-machines-081846ea6753); [Conceptual taxonomy of AI agents](https://www.sciencedirect.com/science/article/pii/S1566253525006712)).

## 1. Introduction

AI agents and autonomous systems—spanning software‑only LLM‑based agents, multi‑agent systems, and embodied robots—have become central to current visions of AI‑enabled transformation across industries and public services ([Springer survey on agentic AI architectures](https://link.springer.com/article/10.1007/s10462-025-11422-4); [MIT AeroAstro autonomous systems](https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/)). Recent surveys of agentic AI highlight a move from static model prompting toward architectures that integrate planning, tool use, memory, perception, and feedback, enabling more autonomous goal pursuit over longer horizons ([Top AI agent research round‑ups](https://levelup.gitconnected.com/the-top-10-ai-agent-research-papers-of-2025-key-takeaways-and-how-you-can-apply-them-17c4e29ed726); [Google Cloud guidance on agentic apps](https://medium.com/google-cloud/getting-ready-for-a-wave-of-agentic-apps-in-2025-check-out-these-articles-89b605c41e54)). In parallel, autonomous systems research in fields such as manufacturing, mobility, and aerospace has focused on robust decision‑making under uncertainty, real‑time control, and safe operation in open‑world environments ([Decision‑making in autonomous vehicles](http://www.coscipress.com/api/file/download/20251118/1716055e0587110-5dc1-4cdb-b3e9-dea19f86486d/Evolution%20of%20Decision-making%20Systems%20in%20Autonomous%20Vehicle%20Technology%20-%20A%20Comprehensive%20Review.pdf); [ResearchGate review on autonomous systems](https://www.researchgate.net/publication/386099540_The_Evolution_of_AI_in_Autonomous_Systems_Innovations_Challenges_and_Future_Prospects)).

The stakes of these developments are high because AI agents increasingly mediate knowledge work, scientific discovery, logistics, and safety‑critical operations, raising questions about productivity, resilience, and accountability ([McKinsey state of AI 2025](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai); [xCube Labs on real‑world agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/); [IQVIA healthcare agents](https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare)). Market studies estimate strong growth in autonomous AI and agent markets and highlight their role in competitive advantage across sectors, placing them alongside data infrastructure and cloud platforms as strategic assets ([Global Market Insights on autonomous AI and agents](https://www.gminsights.com/industry-analysis/autonomous-ai-and-autonomous-agents-market); [Microsoft AI Economy Institute on global AI adoption](https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/); [AI Infrastructure 2025](https://enkiai.com/ai-market-intelligence/ai-infrastructure-2025-master-your-cloud-gpu-strategy)). At the same time, workforce and societal studies warn of significant distributional effects, with benefits and harms likely to differ across occupations, firms, and regions ([BCG AI at Work 2025](https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain); [UNESCO societal implications](https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight%20from%20Signals.pdf); [IMF on uneven global AI impact](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)).

This report synthesizes findings from recent research papers, technical benchmarks, industry white papers, and policy analyses published in 2024–2025 to answer the overarching question: what are the latest breakthroughs in AI agents and autonomous systems in this period, and what do they imply for technical practice, economic organization, and governance ([Top AI agent paper round‑ups](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [Stanford AI Index 2025](https://hai.stanford.edu/ai-index/2025-ai-index-report/technical-performance); [World Economic Forum on AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/); [Dentons on AI regulation trends](https://www.dentons.com/en/insights/articles/2025/january/10/ai-trends-for-2025-ai-regulation-governance-and-ethics)). The scope follows three pillars: (1) technical advances in architectures, decision‑making, evaluation, infrastructure, and hardware; (2) applications and real‑world impact across priority domains; and (3) governance, risks, and ecosystem dynamics including open‑source versus proprietary models and regional regulatory divergence ([research task framing](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [McKinsey on open‑source in AI](https://www.mckinsey.com/~/media/mckinsey/business%20functions/quantumblack/our%20insights/open%20source%20technology%20in%20the%20age%20of%20ai/open-source-technology-in-the-age-of-ai_final.pdf); [OECD on emerging divides in AI](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-artificial-intelligence_eeb5e120/7376c776-en.pdf)).

## 2. Key Findings

### 2.1 Evolution of AI Agent Architectures and Decision‑Making

Recent literature and curated lists of leading AI‑agent papers point to a rapid maturation of agentic architectures built on large language models and related foundation models ([Top AI agent papers, Feb 2025](https://www.linkedin.com/pulse/must-read-alert-top-10-ai-agent-research-papers-first-anshuman-jha-vd8zc); [Analytics Vidhya top AI agent papers](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [Level Up Coding overview](https://levelup.gitconnected.com/the-top-10-ai-agent-research-papers-of-2025-key-takeaways-and-how-you-can-apply-them-17c4e29ed726); [PaperDigest influential arXiv AI papers](https://www.paperdigest.org/2025/03/most-influential-arxiv-artificial-intelligence-papers-2025-03-version/)). According to a comprehensive survey of agentic AI architectures, modern systems increasingly decompose tasks into planning, tool‑calling, memory, and reflection components, enabling them to operate over extended horizons and adapt to partial feedback rather than respond to isolated prompts ([Springer survey on agentic AI architectures](https://link.springer.com/article/10.1007/s10462-025-11422-4)). Framework‑level guidance from platform providers similarly emphasizes modular compositions of single and multi‑agent patterns for orchestration, error handling, and specialization ([Google Cloud on agentic apps](https://medium.com/google-cloud/getting-ready-for-a-wave-of-agentic-apps-in-2025-check-out-these-articles-89b605c41e54); [Best open‑source AI platforms](https://greennode.ai/blog/best-open-source-ai-platforms)).

Across these works, a recurring theme is the use of tools and APIs—such as retrieval systems, code execution sandboxes, search, and enterprise application connectors—to ground LLM outputs and support reliable action in complex environments ([Top AI agent papers, Feb 2025](https://www.linkedin.com/pulse/must-read-alert-top-10-ai-agent-research-papers-first-anshuman-jha-vd8zc); [Best AI agent evaluation benchmarks](https://o-mega.ai/articles/the-best-ai-agent-evals-and-benchmarks-full-2025-guide); [xCube Labs real‑world agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/)). Multi‑agent systems are increasingly used to decompose problems, model diverse roles (such as planner, executor, and critic), and enable negotiation or redundancy, particularly in code generation, operations, and research‑assistant use cases ([Analytics Vidhya top AI agent papers](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/); [Level Up Coding overview](https://levelup.gitconnected.com/the-top-10-ai-agent-research-papers-of-2025-key-takeaways-and-how-you-can-apply-them-17c4e29ed726)).

In decision‑making and control, reviews in domains such as autonomous vehicles and cyber‑physical systems highlight movement from rule‑based or hand‑engineered stacks toward learning‑enabled perception and planning modules that aim to improve sample efficiency and robustness under distribution shift ([Decision‑making in autonomous vehicles](http://www.coscipress.com/api/file/download/20251118/1716055e0587110-5dc1-4cdb-b3e9-dea19f86486d/Evolution%20of%20Decision-making%20Systems%20in%20Autonomous%20Vehicle%20Technology%20-%20A%20Comprehensive%20Review.pdf); [ResearchGate review on autonomous systems](https://www.researchgate.net/publication/386099540_The_Evolution_of_AI_in_Autonomous_Systems_Innovations_Challenges_and_Future_Prospects)). Overviews of autonomous decision‑making describe AI agents that learn from high‑dimensional data, use reinforcement learning and deep planning, and operate with partial autonomy while humans retain oversight, especially in safety‑critical contexts ([Boston Institute of Analytics on autonomous decision‑making](https://bostoninstituteofanalytics.org/blog/ai-agents-and-machine-learning-the-2025-evolution-of-autonomous-decision-making/); [MIT AeroAstro autonomous systems](https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/); [The Rise of Autonomous Systems](https://medium.com/@cevherd/the-rise-of-autonomous-systems-revolutionizing-the-future-19fe1d75a628)).

### 2.2 Advances in Embodied Autonomy and Cyber‑Physical Systems

Reports on manufacturing, mobility, and robotics indicate that autonomous systems are increasingly deployed in hybrid human‑machine configurations, where AI handles perception, optimization, and low‑level control while humans supervise, intervene in edge cases, and make strategic decisions ([Evolution of manufacturing with autonomous systems](https://www.linkedin.com/pulse/evolution-manufacturing-autonomous-systems-2025-maxwell-almeida-fo9ye); [MIT AeroAstro autonomous systems](https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/)). Analyses of human‑interactive manufacturing emphasize that autonomy is being used to reconfigure workflows, reduce downtime, and enable mass customization, rather than to fully eliminate human labor, although longer‑term substitution effects remain debated ([Evolution of manufacturing with autonomous systems](https://www.linkedin.com/pulse/evolution-manufacturing-autonomous-systems-2025-maxwell-almeida-fo9ye); [UNESCO societal implications](https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight%20from%20Signals.pdf)).

In autonomous driving, new datasets—including multi‑sensor and 4D radar datasets—support training and evaluation of perception and planning systems under more varied conditions, expanding beyond earlier benchmarks focused on limited sensor configurations or geographies ([BasicAI on new autonomous driving datasets](https://www.basic.ai/blog-post/15-new-autonomous-driving-datasets-in-2024-2025); [Decision‑making in autonomous vehicles](http://www.coscipress.com/api/file/download/20251118/1716055e0587110-5dc1-4cdb-b3e9-dea19f86486d/Evolution%20of%20Decision-making%20Systems%20in%20Autonomous%20Vehicle%20Technology%20-%20A%20Comprehensive%20Review.pdf)). Reviews of AI in autonomous systems describe progress in integrating advanced sensors, data analytics, and learning‑based decision‑making, enabling more adaptive responses to dynamic environments and partially addressing challenges such as occlusions, adverse weather, and complex traffic interactions ([The Rise of Autonomous Systems](https://medium.com/@cevherd/the-rise-of-autonomous-systems-revolutionizing-the-future-19fe1d75a628); [ResearchGate review on autonomous systems](https://www.researchgate.net/publication/386099540_The_Evolution_of_AI_in_Autonomous_Systems_Innovations_Challenges_and_Future_Prospects)).

More broadly, surveys of autonomous systems in aerospace and defense describe embodied intelligent agents such as drones and cooperative fleets that operate under constraints of latency, communication reliability, and contested environments, often combining on‑board intelligence with ground‑based coordination ([MIT AeroAstro autonomous systems](https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/); [Hardware for HPC and AI 2025–2035](https://www.idtechex.com/en/research-report/hardware-for-hpc-and-ai-2025/1058)). These developments are underpinned by advances in edge and embedded AI hardware, including low‑power accelerators and TinyML approaches that allow some perception and control to be executed locally on devices ranging from drones to consumer gadgets ([AI hardware market analysis](https://www.gminsights.com/industry-analysis/ai-hardware-market); [Making Next Gadget on AI hardware](https://www.youtube.com/watch?v=CKngyrU7md8); [How AI computing hardware works](https://www.linkedin.com/pulse/how-ai-computing-hardware-works-one-simple-flow-2025-hivmc/)).

### 2.3 Benchmarks and Evaluation of AI Agents

Benchmarking has become a focal point for both model and agent research, with new suites introduced to measure complex reasoning, tool use, and real‑world task completion. The 2025 edition of the Stanford AI Index documents substantial performance gains on broad benchmarks such as MMMU, GPQA, and SWE‑bench, which test multi‑disciplinary reasoning, graduate‑level question answering, and software engineering capabilities respectively ([Stanford AI Index 2025](https://hai.stanford.edu/ai-index/2025-ai-index-report/technical-performance)). Guides to AI‑agent evaluation emphasize that such benchmarks increasingly go beyond static question‑answering to environments where agents must plan, call tools, interact with user interfaces, and recover from failures ([Best AI agent evaluation benchmarks](https://o-mega.ai/articles/the-best-ai-agent-evals-and-benchmarks-full-2025-guide); [Benchmarking AI agents in 2025](https://metadesignsolutions.com/benchmarking-ai-agents-in-2025-top-tools-metrics-performance-testing-strategies/)).

Collections of agent benchmarks highlight tasks such as web navigation, API‑based operations, and multi‑step business processes, with metrics that include success rate, robustness under perturbations, and cost‑ or latency‑aware efficiency ([Evidently AI agent benchmarks](https://www.evidentlyai.com/blog/ai-agent-benchmarks); [Best AI agent evaluation benchmarks](https://o-mega.ai/articles/the-best-ai-agent-evals-and-benchmarks-full-2025-guide)). Industry and research commentaries caution, however, that selecting benchmarks that generalize to real‑world deployments remains challenging, given the tendency for models and agents to overfit to specific test suites and the difficulty of capturing long‑tail failure modes ([Zeta Alpha on evaluating AI systems](https://www.zeta-alpha.com/post/trends-in-ai-may-25); [Benchmarking AI agents in 2025](https://metadesignsolutions.com/benchmarking-ai-agents-in-2025-top-tools-metrics-performance-testing-strategies/)).

In addition to technical benchmarks, organizations increasingly adopt domain‑specific evaluation protocols that combine automated metrics, scenario‑based testing, and human evaluation, particularly in regulated sectors like healthcare and finance where reliability and fairness are critical ([IQVIA healthcare agents](https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare); [Cloud Security Alliance on AI and privacy](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal-developments); [AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/)). Compliance‑oriented guidance around the EU AI Act and similar frameworks emphasizes traceability, bias testing, and ongoing monitoring for high‑risk systems, effectively linking benchmarking practices to regulatory obligations ([Artificial Intelligence 2025 – Chambers guide](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/); [TrustCloud governance guidance](https://community.trustcloud.ai/docs/grc-launchpad/grc-101/governance/data-privacy-and-ai-ethical-considerations-and-best-practices/)).

### 2.4 Infrastructure, Hardware, and Tooling Ecosystems

Breakthroughs in AI agents are tightly coupled to advances in infrastructure and hardware, which determine feasible model sizes, latency, and deployment topologies. Analyses of AI infrastructure in 2025 underscore the centrality of large‑scale GPU and accelerator clusters, high‑bandwidth networking, and optimized data pipelines for training and serving AI‑native decision systems, often delivered via hyperscale clouds ([AI Infrastructure 2025](https://enkiai.com/ai-market-intelligence/ai-infrastructure-2025-master-your-cloud-gpu-strategy); [S&P Global on AI infrastructure](https://www.spglobal.com/market-intelligence/en/news-insights/research/2025/10/ai-infrastructure-midyear-2025-update-and-future-technology-considerations); [CRA Quad Paper on foundational AI infrastructure](https://cra.org/wp-content/uploads/2025/04/2024-2025-CRA-Quad-Paper_-Now-More-Than-Ever-Foundational-AI-Research-Infrastructure-Depends-on-the-Federal-Government.pdf)). Global adoption reports tie leading national AI performance to a combination of digital infrastructure, data availability, and workforce skills, suggesting that agentic systems both depend on and reinforce broader digitalization trends ([Microsoft AI Economy Institute global AI adoption](https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/); [IMF on global AI impact](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)).

Hardware‑focused studies describe an expanding market for AI accelerators in data centers and at the edge, with NVIDIA maintaining a strong position while cloud providers and semiconductor firms introduce custom chips tailored to AI workloads, and new memory technologies targeting bandwidth and energy constraints ([Deloitte on AI hardware](https://www.deloitte.com/us/en/insights/focus/tech-trends/2025/tech-trends-ai-hardware-and-computation-leading-ai-revolution.html); [AI hardware market analysis](https://www.gminsights.com/industry-analysis/ai-hardware-market); [IDTechEx hardware report](https://www.idtechex.com/en/research-report/hardware-for-hpc-and-ai-2025/1058)). Commentaries on AI computing flows explain how these hardware trends underpin capabilities such as real‑time inference for robotic control and on‑device personalization, while also raising concerns about concentration of compute resources and environmental impact ([How AI computing hardware works](https://www.linkedin.com/pulse/how-ai-computing-hardware-works-one-simple-flow-2025-hivmc/); [Making Next Gadget on AI hardware](https://www.youtube.com/watch?v=CKngyrU7md8)).

On the software side, open‑source and proprietary ecosystems both contribute to the agent tooling stack. Analyses of open‑source technology in the age of AI argue that open frameworks, models, and orchestration tools accelerate experimentation and reduce costs, especially for smaller organizations, even as proprietary platforms retain advantages in integration, scale, and support ([McKinsey on open‑source in AI](https://www.mckinsey.com/~/media/mckinsey/business%20functions/quantumblack/our%20insights/open%20source%20technology%20in%20the%20age%20of%20ai/open-source-technology-in-the-age-of-ai_final.pdf); [PlanetCrust on open vs proprietary software](https://www.planetcrust.com/open-source-software-v-proprietary-software-2025/?utm_campaign=blog)). Commentaries on AI models specifically frame competition between open‑source and proprietary AI as a dynamic race, with open models closing performance gaps while proprietary systems leverage tight integration with cloud, data, and tooling ecosystems ([Open‑source vs proprietary AI models](https://medium.com/@simplenight/open-source-vs-proprietary-ai-models-whos-winning-the-race-in-2025-1370ef81e4bc); [Best open‑source AI platforms](https://greennode.ai/blog/best-open-source-ai-platforms); [Stack Overflow on open‑source AI and younger developers](https://stackoverflow.blog/2025/04/07/open-source-ai-are-younger-developers-leading-the-way/)).

### 2.5 Applications, Economic Impact, and Human‑AI Collaboration

Use‑case surveys document a broadening set of domains where AI agents are deployed in production, including customer support, marketing, software development, fraud detection, supply‑chain optimization, healthcare operations, and scientific research support ([xCube Labs real‑world agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/); [Vouched real‑life AI agents](https://www.vouched.id/learn/best-of-2025-real-life-ai-agents-are-at-work-in-these-fields); [LinkedIn on 2024–2025 AI agent market trends](https://www.linkedin.com/posts/rakeshgohel01_2024-ai-agent-market-trends-are-now-2025-activity-7298348330613387264-W0_p)). In healthcare and life sciences, providers and vendors report using agentic systems to orchestrate data retrieval, assist clinicians with documentation, analyze trial data, and explore drug repurposing opportunities, often under human oversight and within narrowly scoped workflows ([IQVIA healthcare agents](https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare); [Agentic AI in healthcare – TATEEDA](https://tateeda.com/blog/agentic-ai-in-healthcare-trends-and-types); [Agentic AI in healthcare – Kellton](https://www.kellton.com/kellton-tech-blog/agentic-ai-healthcare-trends-2026); [Future of AI agents in business and society](https://www.linkedin.com/pulse/future-ai-agents-transforming-business-society-2025-beyond-john-enoh-hjo6c)).

Statistics compiled across surveys and vendor reports indicate rapid growth in AI‑agent adoption and perceived productivity impact, with organizations reporting efficiency gains, faster time‑to‑insight, and new forms of automation, though realized benefits vary by maturity and domain ([Datagrid AI agent statistics](https://www.datagrid.com/blog/ai-agent-statistics); [IBM expectations vs reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality); [McKinsey state of AI 2025](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai); [Microsoft AI Economy Institute global AI adoption](https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/)). Market forecasts for autonomous AI and agents project substantial revenue growth and note that a large majority of enterprises either have pilot projects underway or plan to incorporate autonomous capabilities into operations by the mid‑2020s ([Global Market Insights on autonomous AI and agents](https://www.gminsights.com/industry-analysis/autonomous-ai-and-autonomous-agents-market); [emagine on AI‑agent trends](https://www.emagine.org/blogs/ai-agents-lead-the-way-key-trends-shaping-2025); [Datagrid AI agent statistics](https://www.datagrid.com/blog/ai-agent-statistics)).

Economy‑wide analyses suggest that AI and emerging technologies will substantially influence growth and productivity, with differentiated impacts across sectors depending on task structure, capital intensity, and regulatory exposure ([ETA economic impact report](https://www.joineta.org/blog/economic-impact-and-projections-report-the-integration-of-ai-and-emerging-technologies-2025-2030); [KPMG on generative AI and growth](https://kpmg.com/kpmg-us/content/dam/kpmg/pdf/2025/gen-ai-economic-growth.pdf); [IMF on the global impact of AI](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)). Workforce‑oriented studies highlight both new roles—such as AI product owners, prompt and workflow engineers, and human‑in‑the‑loop supervisors—and risks including displacement of routine cognitive tasks, erosion of entry‑level opportunities, and increased pressure on mid‑skill roles ([BCG AI at Work 2025](https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain); [World Economic Forum on AI and jobs](https://www.weforum.org/stories/2025/04/ai-jobs-international-workers-day/); [National Fund for Workforce Solutions on AI and work](https://nationalfund.org/ai-and-the-future-of-work/); [TIAA Institute on AI and work](https://www.tiaa.org/content/dam/tiaa/institute/pdf/insights-report/2025-10/tiaa-institute-insights-brief-ai-and-the-future-of-work-reshaping-the-landscape-of-human-work-watson-mastry-aug-2025.pdf)).

Studies of human‑AI collaboration find that, despite the rhetoric of “co‑pilots,” many current systems function more as advanced decision‑support tools than as truly collaborative partners, with humans often adapting their workflows to AI system affordances rather than vice versa ([Human‑AI collaboration taxonomy](https://www.frontiersin.org/journals/computer-science/articles/10.3389/fcomp.2024.1521066/full); [Taxonomy of interaction patterns](https://pure.johnshopkins.edu/en/publications/human-ai-collaboration-is-not-very-collaborative-yet-a-taxonomy-o); [Human‑AI interaction in creative domains](https://dl.acm.org/doi/10.1145/3743049.3743081)). Additional work on human‑AI co‑creation and interaction strategies highlights the importance of control mechanisms, transparency, and context‑sensitive interfaces to support trust and effective collaboration in creative, analytical, and operational tasks ([Exploring collaboration patterns in human‑AI co‑creation](https://arxiv.org/pdf/2507.06000); [Human‑AI interaction strategies](https://www.emergentmind.com/topics/human-ai-interaction-strategies)).

### 2.6 Governance, Risks, and Geographical Differences

Regulatory and governance frameworks for AI agents are evolving rapidly, with a notable shift toward risk‑based approaches that classify systems by potential harm and impose graduated requirements for transparency, data governance, and oversight. Analyses of global regulatory trends describe how the EU’s AI Act is shaping international discourse through its focus on high‑risk use cases, conformity assessments, and post‑market monitoring, influencing firms far beyond Europe’s borders ([Artificial Intelligence 2025 – Chambers guide](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [Dentons on AI regulation and ethics](https://www.dentons.com/en/insights/articles/2025/january/10/ai-trends-for-2025-ai-regulation-governance-and-ethics); [AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/)). U.S. and international overviews highlight emerging legislation and guidance that address algorithmic discrimination, consumer protection, and safety, including state laws such as Colorado’s AI statute, as well as sector‑specific rules in finance and healthcare ([CRS report on regulating AI](https://www.congress.gov/crs-product/R48555); [GARP on AI regulation by geography](https://www.garp.org/risk-intelligence/culture-governance/ai-regulation-geography-240712); [Cloud Security Alliance on AI and privacy](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal-developments)).

Corporate responsible‑AI frameworks complement formal regulations by providing principles and operational guidance on fairness, transparency, privacy, and security in AI tools and agents, often codified in internal policies, tooling, and review processes ([Microsoft Responsible AI](https://www.microsoft.com/en-us/ai/responsible-ai); [TrustCloud governance guidance](https://community.trustcloud.ai/docs/grc-launchpad/grc-101/governance/data-privacy-and-ai-ethical-considerations-and-best-practices/); [Medium on AI ethics in 2025](https://medium.com/@mmp3071/ai-ethics-in-2025-balancing-innovation-with-responsibility-4a267f1b7717)). Discussions of AI compliance in 2025 emphasize requirements for bias testing, documentation, and monitoring in high‑risk systems, including many agentic and autonomous applications in finance, employment, and critical infrastructure ([AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/); [Cloud Security Alliance on AI and privacy](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal-developments)).

Despite these developments, research on risks and limitations points to unresolved issues around accountability, unpredictability, and systemic impacts. The World Economic Forum’s analysis of AI agents highlights ethical questions about delegating decisions to agents whose internal reasoning is opaque and whose objectives may drift during operation ([World Economic Forum on AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/)). A conceptual taxonomy of AI agents and agentic AI catalogues failure modes and risk types across domains, arguing that risk is shaped not only by capability level but also by the agent’s degree of autonomy, access to tools, and integration into socio‑technical systems ([Conceptual taxonomy of AI agents](https://www.sciencedirect.com/science/article/pii/S1566253525006712)). Red‑teaming and safety analyses for generative AI agents in autonomous machines further identify challenges such as catastrophic forgetting, real‑time processing constraints, distribution shift, and the lack of formal safety guarantees, particularly in physical environments ([Safety challenges for generative AI agents](https://cobusgreyling.medium.com/safety-challenges-for-generative-ai-agents-in-autonomous-machines-081846ea6753); [ResearchGate review on autonomous systems](https://www.researchgate.net/publication/386099540_The_Evolution_of_AI_in_Autonomous_Systems_Innovations_Challenges_and_Future_Prospects)).

Some authors explicitly argue that fully autonomous, open‑ended AI agents that act without human oversight should not be developed with current technologies, given uncertainties about alignment, controllability, and systemic risk (["Fully Autonomous AI Agents Should Not be Developed"](https://arxiv.org/html/2502.02649v3); [World Economic Forum on AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/)). Business‑oriented analyses of the adoption of AI and agentic systems similarly highlight vendor lock‑in, observability challenges, and difficulties in auditability and explanation as key barriers to confident large‑scale deployment, especially in regulated and safety‑critical settings ([Adoption of AI and agentic systems](https://cmr.berkeley.edu/2025/08/adoption-of-ai-and-agentic-systems-value-challenges-and-pathways/); [AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/)). At a macro level, organizations such as the OECD document emerging divides in AI adoption and capability between countries, firms, and sectors, suggesting that governance outcomes will also be shaped by underlying economic and institutional disparities ([OECD on emerging AI divides](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-artificial-intelligence_eeb5e120/7376c776-en.pdf); [IMF on uneven global AI impact](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf); [Microsoft AI Economy Institute global AI adoption](https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/)).

## 3. Analysis and Implications

Taken together, the technical literature and industry experience suggest a convergence on agentic architectures that treat large models as components within broader decision‑making systems rather than as standalone intelligences, with planning, tool use, and feedback loops providing structure and guardrails ([Springer survey on agentic AI architectures](https://link.springer.com/article/10.1007/s10462-025-11422-4); [Google Cloud on agentic apps](https://medium.com/google-cloud/getting-ready-for-a-wave-of-agentic-apps-in-2025-check-out-these-articles-89b605c41e54); [Top AI agent research overviews](https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/)). This design pattern mitigates some limitations of general‑purpose models by grounding them in tools and data, but it also introduces new failure modes at the system level—for example, when planning, tool‑calling, and monitoring components interact in unanticipated ways, as highlighted by taxonomies of interaction patterns and risk analyses of generative agents ([Conceptual taxonomy of AI agents](https://www.sciencedirect.com/science/article/pii/S1566253525006712); [Safety challenges for generative AI agents](https://cobusgreyling.medium.com/safety-challenges-for-generative-ai-agents-in-autonomous-machines-081846ea6753)).

The rapid proliferation of benchmarks helps structure innovation and claims of progress, yet comparisons across systems remain difficult because evaluation environments differ in realism, coverage of edge cases, and alignment with deployment contexts ([Best AI agent evaluation benchmarks](https://o-mega.ai/articles/the-best-ai-agent-evals-and-benchmarks-full-2025-guide); [Evidently AI agent benchmarks](https://www.evidentlyai.com/blog/ai-agent-benchmarks); [Zeta Alpha on evaluating AI systems](https://www.zeta-alpha.com/post/trends-in-ai-may-25)). Regulatory requirements for documentation, bias testing, and post‑deployment monitoring are likely to push organizations toward more rigorous, domain‑specific evaluation pipelines that combine simulation, red‑teaming, and human review, which in turn may favor larger firms with the resources to build such infrastructure ([Artificial Intelligence 2025 – Chambers guide](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [AI compliance in 2025](https://vodworks.com/blogs/ai-compliance/); [Cloud Security Alliance on AI and privacy](https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal-developments)).

Infrastructure and hardware trends imply that compute access will remain a key lever shaping who can build and deploy cutting‑edge agentic systems. Concentration of high‑end accelerators in a few technology firms and regions, combined with rising capital intensity, risks exacerbating existing divides in AI capability and adoption documented by international organizations, even as open‑source models and more efficient hardware architectures try to broaden access ([AI Infrastructure 2025](https://enkiai.com/ai-market-intelligence/ai-infrastructure-2025-master-your-cloud-gpu-strategy); [AI hardware market analysis](https://www.gminsights.com/industry-analysis/ai-hardware-market); [OECD on emerging AI divides](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-artificial-intelligence_eeb5e120/7376c776-en.pdf); [IMF on uneven global AI impact](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)). Analyses of open‑source versus proprietary AI suggest that while open ecosystems drive experimentation and local adaptation, proprietary stacks remain attractive for mission‑critical uses that demand integration, security, and support, reinforcing a dual‑track innovation landscape ([Open‑source vs proprietary AI models](https://medium.com/@simplenight/open-source-vs-proprietary-ai-models-whos-winning-the-race-in-2025-1370ef81e4bc); [McKinsey on open‑source in AI](https://www.mckinsey.com/~/media/mckinsey/business%20functions/quantumblack/our%20insights/open%20source%20technology%20in%20the%20age%20of%20ai/open-source-technology-in-the-age-of-ai_final.pdf); [Best open‑source AI platforms](https://greennode.ai/blog/best-open-source-ai-platforms)).

From an organizational perspective, case studies and surveys show that AI agents are beginning to reshape workflows and organizational structures, but realized productivity gains depend heavily on process redesign, change management, and human‑AI interaction design rather than on agent technology alone ([xCube Labs real‑world agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/); [IBM expectations vs reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality); [Human‑AI collaboration taxonomy](https://www.frontiersin.org/journals/computer-science/articles/10.3389/fcomp.2024.1521066/full)). Studies of collaboration patterns suggest that without careful design, agents can either overwhelm users with low‑quality suggestions or encourage over‑reliance and deskilling, reinforcing the importance of transparency, controllability, and appropriate division of labor between humans and AI systems ([Exploring collaboration patterns in human‑AI co‑creation](https://arxiv.org/pdf/2507.06000); [Human‑AI interaction strategies](https://www.emergentmind.com/topics/human-ai-interaction-strategies); [BCG AI at Work 2025](https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain)).

At the societal level, economic and labor market analyses indicate that AI agents and autonomous systems will likely amplify existing inequalities unless accompanied by substantial investments in skills, social protection, and inclusive innovation policies. Reports from multilateral organizations and think tanks warn that countries and firms with limited access to data, compute, and complementary capabilities may fall further behind, while within economies, lower‑income and routine‑task workers face higher displacement risks, even as new high‑skill opportunities emerge ([UNESCO societal implications](https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight%20from%20Signals.pdf); [IMF on the global impact of AI](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf); [World Economic Forum on AI and jobs](https://www.weforum.org/stories/2025/04/ai-jobs-international-workers-day/); [National Fund for Workforce Solutions on AI and work](https://nationalfund.org/ai-and-the-future-of-work/)). Governance debates about whether fully autonomous agents should be developed at all, or only under strict constraints, reflect broader concerns about alignment, systemic risk, and democratic control over critical infrastructures, particularly as agentic capabilities continue to scale (["Fully Autonomous AI Agents Should Not be Developed"](https://arxiv.org/html/2502.02649v3); [World Economic Forum on AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/); [Adoption of AI and agentic systems](https://cmr.berkeley.edu/2025/08/adoption-of-ai-and-agentic-systems-value-challenges-and-pathways/)).

## 4. Limitations

The evidence synthesized in this report is drawn from a curated set of peer‑reviewed articles, preprints, industry white papers, vendor case studies, and policy reports from 2024–2025, which, while diverse, do not constitute a systematic review of all work on AI agents and autonomous systems. Many of the sources are forward‑looking market or technology analyses whose projections—such as market size estimates and adoption rates—may reflect optimistic scenarios or particular methodological assumptions, as is evident in varying forecasts across reports from consultancies and international organizations ([Global Market Insights on autonomous AI and agents](https://www.gminsights.com/industry-analysis/autonomous-ai-and-autonomous-agents-market); [ETA economic impact report](https://www.joineta.org/blog/economic-impact-and-projections-report-the-integration-of-ai-and-emerging-technologies-2025-2030); [IMF on the global impact of AI](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)).

There is also a regional and sectoral bias in the available literature toward high‑income economies and technology‑intensive industries, as highlighted by analyses of emerging divides in AI adoption and capability ([OECD on emerging AI divides](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-artificial-intelligence_eeb5e120/7376c776-en.pdf); [Microsoft AI Economy Institute global AI adoption](https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/)). Technical details of proprietary systems, including many commercial agent frameworks, remain opaque, limiting independent verification of performance and safety claims and relying instead on secondary analyses and high‑level descriptions ([IBM expectations vs reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality); [Open‑source vs proprietary AI models](https://medium.com/@simplenight/open-source-vs-proprietary-ai-models-whos-winning-the-race-in-2025-1370ef81e4bc)). Finally, the rapidly evolving nature of the field means that capabilities, benchmarks, and regulatory regimes are likely to change over short time horizons, reducing the durability of any static snapshot such as this one ([Stanford AI Index 2025](https://hai.stanford.edu/ai-index/2025-ai-index-report/technical-performance); [Dentons on AI regulation trends](https://www.dentons.com/en/insights/articles/2025/january/10/ai-trends-for-2025-ai-regulation-governance-and-ethics)).

## 5. Conclusion

Recent breakthroughs in AI agents and autonomous systems between 2024 and 2025 are characterized less by a single transformative invention than by the consolidation and scaling of agentic design patterns, benchmarks, and infrastructure that make it feasible to embed AI into complex workflows and physical systems. Research on agent architectures, benchmarks, and embodied autonomy shows that LLM‑based agents, multi‑agent frameworks, and learning‑enabled control systems are now capable of solving longer‑horizon, more complex tasks in both digital and physical domains, albeit with important reliability and safety caveats ([Springer survey on agentic AI architectures](https://link.springer.com/article/10.1007/s10462-025-11422-4); [Decision‑making in autonomous vehicles](http://www.coscipress.com/api/file/download/20251118/1716055e0587110-5dc1-4cdb-b3e9-dea19f86486d/Evolution%20of%20Decision-making%20Systems%20in%20Autonomous%20Vehicle%20Technology%20-%20A%20Comprehensive%20Review.pdf); [Best AI agent evaluation benchmarks](https://o-mega.ai/articles/the-best-ai-agent-evals-and-benchmarks-full-2025-guide)).

Empirical deployments across sectors indicate that AI agents can deliver tangible productivity and service‑quality improvements when combined with process redesign and robust governance, but they also expose integration challenges, uneven benefits, and risks of workforce disruption and over‑reliance on opaque systems ([xCube Labs real‑world agents](https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/); [IBM expectations vs reality](https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality); [BCG AI at Work 2025](https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain); [UNESCO societal implications](https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight%20from%20Signals.pdf)). Governance and safety research stresses that current capabilities do not yet justify unbounded autonomy and calls for careful scoping, human oversight, and robust evaluation, alongside institutional reforms to address distributional and geopolitical consequences of uneven AI adoption ([World Economic Forum on AI agent risks](https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/); ["Fully Autonomous AI Agents Should Not be Developed"](https://arxiv.org/html/2502.02649v3); [OECD on emerging AI divides](https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-artificial-intelligence_eeb5e120/7376c776-en.pdf)).

Looking ahead, the central questions for AI agents and autonomous systems are likely to revolve around reliability, alignment, and equitable access rather than raw capability alone. Continued progress will depend on advances in safety‑aware architectures, evaluation regimes that more closely mirror real‑world stakes, and governance frameworks that enable innovation while constraining systemic risks, as emphasized across technical, economic, and policy analyses reviewed here ([Safety challenges for generative AI agents](https://cobusgreyling.medium.com/safety-challenges-for-generative-ai-agents-in-autonomous-machines-081846ea6753); [Stanford AI Index 2025](https://hai.stanford.edu/ai-index/2025-ai-index-report/technical-performance); [Artificial Intelligence 2025 – Chambers guide](https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025); [IMF on the global impact of AI](https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)). Under such conditions, AI agents and autonomous systems could become a foundational layer of digital and physical infrastructure, shaping economic organization and everyday life in ways comparable to past general‑purpose technologies, while remaining subject to ongoing contestation and oversight ([ETA economic impact report](https://www.joineta.org/blog/economic-impact-and-projections-report-the-integration-of-ai-and-emerging-technologies-2025-2030); [Microsoft AI Economy Institute global AI adoption](https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/); [McKinsey state of AI 2025](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai)).

## References

- "Top 10 AI Agent Research Papers of the First 10 Days of Feb 2025" – Anshuman Jha, LinkedIn (https://www.linkedin.com/pulse/must-read-alert-top-10-ai-agent-research-papers-first-anshuman-jha-vd8zc)
- "Top 10 Research Papers on AI Agents" – Analytics Vidhya (https://www.analyticsvidhya.com/blog/2024/12/ai-agents-research-papers/)
- "Most Influential ArXiv (Artificial Intelligence) Papers (2025-03 Version)" – PaperDigest (https://www.paperdigest.org/2025/03/most-influential-arxiv-artificial-intelligence-papers-2025-03-version/)
- "The Top 10 AI Agent Research Papers of 2025" – Level Up Coding (https://levelup.gitconnected.com/the-top-10-ai-agent-research-papers-of-2025-key-takeaways-and-how-you-can-apply-them-17c4e29ed726)
- "Getting Ready for a Wave of Agentic Apps in 2025" – Google Cloud (https://medium.com/google-cloud/getting-ready-for-a-wave-of-agentic-apps-in-2025-check-out-these-articles-89b605c41e54)
- "The Evolution of Manufacturing: Autonomous Systems and Human-Interactive Manufacturing in 2025" – LinkedIn (https://www.linkedin.com/pulse/evolution-manufacturing-autonomous-systems-2025-maxwell-almeida-fo9ye)
- "Evolution of Decision-making Systems in Autonomous Vehicle Technology" – COSCI Press (http://www.coscipress.com/api/file/download/20251118/1716055e0587110-5dc1-4cdb-b3e9-dea19f86486d/Evolution%20of%20Decision-making%20Systems%20in%20Autonomous%20Vehicle%20Technology%20-%20A%20Comprehensive%20Review.pdf)
- "AI Agents and Machine Learning: The 2025 Evolution of Autonomous Decision-Making" – Boston Institute of Analytics (https://bostoninstituteofanalytics.org/blog/ai-agents-and-machine-learning-the-2025-evolution-of-autonomous-decision-making/)
- "Autonomous Systems & Decision-Making" – MIT AeroAstro (https://aeroastro.mit.edu/research-areas/autonomous-systems-decision-making/)
- "The Rise of Autonomous Systems: Revolutionizing the Future" – Medium (https://medium.com/@cevherd/the-rise-of-autonomous-systems-revolutionizing-the-future-19fe1d75a628)
- "Best AI Agent Evaluation Benchmarks: 2025 Complete Guide" – O-Mega (https://o-mega.ai/articles/the-best-ai-agent-evals-and-benchmarks-full-2025-guide)
- "The 2025 AI Index Report – Technical Performance" – Stanford HAI (https://hai.stanford.edu/ai-index/2025-ai-index-report/technical-performance)
- "Evaluating AI Systems – Trends in AI: May '25" – Zeta Alpha (https://www.zeta-alpha.com/post/trends-in-ai-may-25)
- "Benchmarking AI Agents in 2025: Top Tools, Metrics & Performance" – Metadesign Solutions (https://metadesignsolutions.com/benchmarking-ai-agents-in-2025-top-tools-metrics-performance-testing-strategies/)
- "10 AI Agent Benchmarks" – Evidently AI (https://www.evidentlyai.com/blog/ai-agent-benchmarks)
- "AI Infrastructure 2025: Master Your Cloud & GPU Strategy" – EnkiAI (https://enkiai.com/ai-market-intelligence/ai-infrastructure-2025-master-your-cloud-gpu-strategy)
- "Tech Updates of 2025: AI Breakthroughs & Industry Shifts" – TST Technology (https://tsttechnology.io/blog/tech-updates-of-2025)
- "2024-2025 CRA Quad Paper: Now More Than Ever, Foundational AI Research Infrastructure Depends on the Federal Government" – CRA (https://cra.org/wp-content/uploads/2025/04/2024-2025-CRA-Quad-Paper_-Now-More-Than-Ever-Foundational-AI-Research-Infrastructure-Depends-on-the-Federal-Government.pdf)
- "AI Infrastructure: Midyear 2025 Update and Future Technology Considerations" – S&P Global (https://www.spglobal.com/market-intelligence/en/news-insights/research/2025/10/ai-infrastructure-midyear-2025-update-and-future-technology-considerations)
- "Global AI Adoption in 2025 – AI Economy Institute" – Microsoft (https://www.microsoft.com/en-us/corporate-responsibility/topics/ai-economy-institute/reports/global-ai-adoption-2025/)
- "Hardware Leading the AI Revolution" – Deloitte Insights (https://www.deloitte.com/us/en/insights/focus/tech-trends/2025/tech-trends-ai-hardware-and-computation-leading-ai-revolution.html)
- "AI Hardware Market Size & Share, Statistics Report 2025-2034" – Global Market Insights (https://www.gminsights.com/industry-analysis/ai-hardware-market)
- "Hardware for HPC, Data Centers, and AI 2025-2035" – IDTechEx (https://www.idtechex.com/en/research-report/hardware-for-hpc-and-ai-2025/1058)
- "How AI Computing Hardware Works — In One Simple Flow (2025)" – LinkedIn (https://www.linkedin.com/pulse/how-ai-computing-hardware-works-one-simple-flow-2025-hivmc/)
- "2024 AI Hardware Highlights & 2025 Inspirations" – Making Next Gadget (https://www.youtube.com/watch?v=CKngyrU7md8)
- "The Future of AI Agents: Transforming Business and Society in 2025" – LinkedIn (https://www.linkedin.com/pulse/future-ai-agents-transforming-business-society-2025-beyond-john-enoh-hjo6c)
- "IQVIA Launches New AI Agents for Life Sciences and Healthcare" – IQVIA (https://www.iqvia.com/newsroom/2025/06/iqvia-launches-new-ai-agents-for-life-sciences-and-healthcare)
- "Best of 2025: Real-Life AI Agents Are at Work in These Fields" – Vouched (https://www.vouched.id/learn/best-of-2025-real-life-ai-agents-are-at-work-in-these-fields)
- "Healthcare Agentic AI Trends for 2026" – TATEEDA (https://tateeda.com/blog/agentic-ai-in-healthcare-trends-and-types)
- "Agentic AI in Healthcare: Types, Trends, and 2026 Forecast" – Kellton (https://www.kellton.com/kellton-tech-blog/agentic-ai-healthcare-trends-2026)
- "10 Real-World Examples of AI Agents in 2025" – xCube Labs (https://www.xcubelabs.com/blog/10-real-world-examples-of-ai-agents-in-2025/)
- "AI Agents in 2025: Expectations vs. Reality" – IBM (https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality)
- "2024 AI Agent Market Trends Are Now 2025's New Reality" – LinkedIn (https://www.linkedin.com/posts/rakeshgohel01_2024-ai-agent-market-trends-are-now-2025-activity-7298348330613387264-W0_p)
- "26 AI Agent Statistics (Adoption + Business Impact)" – Datagrid (https://www.datagrid.com/blog/ai-agent-statistics)
- "The Integration of AI and Emerging Technologies (2025-2030)" – ETA (https://www.joineta.org/blog/economic-impact-and-projections-report-the-integration-of-ai-and-emerging-technologies-2025-2030)
- "The State of AI in 2025: Agents, Innovation, and Transformation" – McKinsey (https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai)
- "Generative AI and Economic Growth" – KPMG (https://kpmg.com/kpmg-us/content/dam/kpmg/pdf/2025/gen-ai-economic-growth.pdf)
- "The Global Impact of AI: Mind the Gap, WP/25/76" – IMF (https://www.imf.org/-/media/files/publications/wp/2025/english/wpiea2025076-print-pdf.pdf)
- "Human-AI Collaboration Is Not Very Collaborative Yet: A Taxonomy of Interaction Patterns" – Frontiers in Computer Science (https://www.frontiersin.org/journals/computer-science/articles/10.3389/fcomp.2024.1521066/full)
- "Exploring Collaboration Patterns and Strategies in Human-AI Co-creation" – arXiv (https://arxiv.org/pdf/2507.06000)
- "Human-AI Interaction Patterns in Creative Domains and Their Time" – ACM (https://dl.acm.org/doi/10.1145/3743049.3743081)
- "Human-AI Interaction Strategies" – Emergent Mind (https://www.emergentmind.com/topics/human-ai-interaction-strategies)
- "Societal Implications of AI: Key Insights from Emerging Signals" – UNESCO (https://www.unesco.org/sites/default/files/medias/fichiers/2025/12/Societal%20implications%20of%20AI%20-%20Insight%20from%20Signals.pdf)
- "AI at Work 2025: Momentum Builds, but Gaps Remain" – BCG (https://www.bcg.com/publications/2025/ai-at-work-momentum-builds-but-gaps-remain)
- "Is AI Closing the Door on Entry-Level Job Opportunities?" – World Economic Forum (https://www.weforum.org/stories/2025/04/ai-jobs-international-workers-day/)
- "AI and the Future of Work" – National Fund for Workforce Solutions (https://nationalfund.org/ai-and-the-future-of-work/)
- "AI and the Future of Work – TIAA" – TIAA Institute (https://www.tiaa.org/content/dam/tiaa/institute/pdf/insights-report/2025-10/tiaa-institute-insights-brief-ai-and-the-future-of-work-reshaping-the-landscape-of-human-work-watson-mastry-aug-2025.pdf)
- "AI and Privacy: Shifting from 2024 to 2025" – Cloud Security Alliance (https://cloudsecurityalliance.org/blog/2025/04/22/ai-and-privacy-2024-to-2025-embracing-the-future-of-global-legal-developments)
- "Regulating Artificial Intelligence: U.S. and International Approaches" – CRS (https://www.congress.gov/crs-product/R48555)
- "Responsible AI: Ethical Policies and Practices" – Microsoft (https://www.microsoft.com/en-us/ai/responsible-ai)
- "AI Compliance in 2025: Global Regulations, Risks & Best Practices" – Vodworks (https://vodworks.com/blogs/ai-compliance/)
- "Data Privacy & AI Ethics Best Practices" – TrustCloud (https://community.trustcloud.ai/docs/grc-launchpad/grc-101/governance/data-privacy-and-ai-ethical-considerations-and-best-practices/)
- "What Are the Risks and Benefits of 'AI Agents'?" – World Economic Forum (https://www.weforum.org/stories/2024/12/ai-agents-risks-artificial-intelligence/)
- "Fully Autonomous AI Agents Should Not be Developed" – arXiv (https://arxiv.org/html/2502.02649v3)
- "Adoption of AI and Agentic Systems: Value, Challenges, and Pathways" – California Management Review (https://cmr.berkeley.edu/2025/08/adoption-of-ai-and-agentic-systems-value-challenges-and-pathways/)
- "Safety Challenges for Generative AI Agents in Autonomous Machines" – Medium (https://cobusgreyling.medium.com/safety-challenges-for-generative-ai-agents-in-autonomous-machines-081846ea6753)
- "AI Agents vs. Agentic AI: A Conceptual Taxonomy" – Journal article (https://www.sciencedirect.com/science/article/pii/S1566253525006712)
- "Open Source vs Proprietary AI Models: Who's Winning the Race in 2025?" – Medium (https://medium.com/@simplenight/open-source-vs-proprietary-ai-models-whos-winning-the-race-in-2025-1370ef81e4bc)
- "Open Source Technology in the Age of AI" – McKinsey (https://www.mckinsey.com/~/media/mckinsey/business%20functions/quantumblack/our%20insights/open%20source%20technology%20in%20the%20age%20of%20ai/open-source-technology-in-the-age-of-ai_final.pdf)
- "Open-Source Software versus Proprietary Software in 2025" – PlanetCrust (https://www.planetcrust.com/open-source-software-v-proprietary-software-2025/?utm_campaign=blog)
- "Best Open-Source AI Platforms for 2025" – Greennode (https://greennode.ai/blog/best-open-source-ai-platforms)
- "Open-source AI: Are Younger Developers Leading the Way?" – Stack Overflow Blog (https://stackoverflow.blog/2025/04/07/open-source-ai-are-younger-developers-leading-the-way/)
- "AI Trends for 2025: AI Regulation, Governance and Ethics" – Dentons (https://www.dentons.com/en/insights/articles/2025/january/10/ai-trends-for-2025-ai-regulation-governance-and-ethics)
- "Artificial Intelligence 2025" – Chambers Global Practice Guides (https://practiceguides.chambers.com/practice-guides/artificial-intelligence-2025)
- "Emerging Divides in the Transition to Artificial Intelligence" – OECD (https://www.oecd.org/content/dam/oecd/en/publications/reports/2025/06/emerging-divides-in-the-transition-to-artificial-intelligence_eeb5e120/7376c776-en.pdf)
- "AI Regulation by Geography: How Jurisdictional Differences Have Evolved" – GARP (https://www.garp.org/risk-intelligence/culture-governance/ai-regulation-geography-240712)
- "The Evolution of AI in Autonomous Systems: Innovations, Challenges, and Future Prospects" – ResearchGate (https://www.researchgate.net/publication/386099540_The_Evolution_of_AI_in_Autonomous_Systems_Innovations_Challenges_and_Future_Prospects)
- "15 New Autonomous Driving Datasets in 2024–2025" – BasicAI (https://www.basic.ai/blog-post/15-new-autonomous-driving-datasets-in-2024-2025)
- "AI Agents Lead the Way: Key Trends Shaping 2025" – emagine (https://www.emagine.org/blogs/ai-agents-lead-the-way-key-trends-shaping-2025)
- "AI Ethics in 2025: Balancing Innovation with Responsibility" – Medium (https://medium.com/@mmp3071/ai-ethics-in-2025-balancing-innovation-with-responsibility-4a267f1b7717)
- "The Evolution of AI in Autonomous Systems" – Taylor & Francis (https://www.tandfonline.com/doi/full/10.1080/08839514.2025.2463722)
- "Autonomous AI and Autonomous Agents Market" – Global Market Insights (https://www.gminsights.com/industry-analysis/autonomous-ai-and-autonomous-agents-market)


## Architecture Summary

```
┌─────────────────────────────────────────────────────────────┐
│  PHASE 1: PLANNING (with iterative critique)                │
├─────────────────────────────────────────────────────────────┤
│  GPT-4o creates plan → GPT-5.1 critiques → improve → loop   │
│  Output: research_plan.md (approved sub-questions)          │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│  PHASE 2: RESEARCH (gather raw findings)                    │
├─────────────────────────────────────────────────────────────┤
│  GPT-4o executes Tavily searches for each sub-question      │
│  Output: research_plan.md (with raw findings + URLs)        │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│  PHASE 3: SYNTHESIS (comprehensive report)                  │
├─────────────────────────────────────────────────────────────┤
│  GPT-5.1 reads all findings and writes professional report  │
│  Output: research_report.md (executive summary, analysis,   │
│          themed sections, implications, citations)          │
└─────────────────────────────────────────────────────────────┘
```

### Why Two Models?

| Model | Role | Why |
|-------|------|-----|
| GPT-4o | Research Agent | Fast, good at tool use, searches |
| GPT-5.1 | Critique + Synthesis | Stronger reasoning, better writing |

## OpenHands SDK Features Used

| Feature | How It's Used |
|---------|--------------|
| **Multi-Agent** | GPT-4o (research) + GPT-5.1 (critique & synthesis) |
| **Iterative Refinement** | Plan critiqued and improved until threshold met |
| **File-based State** | `research_plan.md` → `research_report.md` pipeline |
| **Checkpointing** | Resume from crash via `checkpoint.json` |
| **Observability** | Laminar tracing for debugging (set `LMNR_PROJECT_API_KEY`) |

### Output Files

| File | Purpose |
|------|---------|
| `research_plan.md` | Working document with sub-questions + raw findings |
| `critique.md` | Plan evaluation feedback (debugging) |
| `research_report.md` | **Final deliverable** - comprehensive report |
| `checkpoint.json` | Resume state (auto-deleted on success) |